## Text Mining Project 
#### In this project, I will be exploring *text message data* and creating models to predict if a message is `spam` or not. 

In [20]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('assets/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [4]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### The percentage of the documents in `spam_data` that are spam

In [6]:
def step_zero():
    return spam_data.target.value_counts(normalize=True).iloc[1] * 100
step_zero()

13.406317300789663

### Fitting the training data `X_train` using a Count Vectorizer 
##### then getting the longest token in the vocabulary

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

def step_one():

    vocabulary = CountVectorizer().fit(X_train).vocabulary_
    vocabulary = [x for x in vocabulary.keys()]
    len_vocabulary = [len(x) for x in vocabulary]

    return vocabulary[np.argmax(len_vocabulary)]

step_one()

'com1win150ppmx3age16subscription'

### Fitting a multinomial Naive Bayes classifier model

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def step_two():

    vect = CountVectorizer().fit(X_train)
    
    # Transforming the training data into a document-term matrix
    X_train_vectorized = vect.transform(X_train)
    
    # Calling a Multinomial Naive Bayes classifier
    model = MultinomialNB(alpha = 0.1)
    
    # Model training
    model.fit(X_train_vectorized, y_train)
    
    # Predicting the transformed test documents
    prediction = model.predict_proba(vect.transform(X_test))[:, 1]
    
    # AUC curve
    a = roc_auc_score(y_test, prediction)
    return a 

step_two()

0.991545422134696

### Fiting and transforming the training data `X_train` using a Tfidf Vectorizer

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf among the **max** tf-idf values?

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def step_four():

    tf = TfidfVectorizer().fit(X_train)
    feature_names = np.array(tf.get_feature_names())
    X_train_tf = tf.transform(X_train)
    max_tf_idfs = X_train_tf.max(0).toarray()[0]
    sorted_tf_idxs = max_tf_idfs.argsort()
    sorted_tf_idfs = max_tf_idfs[sorted_tf_idxs]
    smallest_tf_idfs = pd.Series(sorted_tf_idfs[:20], index=feature_names[sorted_tf_idxs[:20]])                    
    largest_tf_idfs = pd.Series(sorted_tf_idfs[-20:][::-1], index=feature_names[sorted_tf_idxs[-20:][::-1]])

    return (smallest_tf_idfs, largest_tf_idfs) 
step_four()

(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 dt

### Fitting and transforming the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly < **3**.

Then fitting a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`

In [10]:
def step_four():

    vect = TfidfVectorizer(min_df = 3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    model = MultinomialNB(alpha = 0.1)
    
    model.fit(X_train_vectorized, y_train)
    
    prediction = model.predict_proba(vect.transform(X_test))[:, 1]

    return roc_auc_score(y_test, prediction) 

step_four()

0.9954968337775665

### Average length of documents (number of characters) for not spam and spam documents

In [11]:
def step_five():
    spam_data['length'] = spam_data['text'].apply(lambda x:len(x))
    return  (np.mean(spam_data['length'][spam_data['target'] == 0]), np.mean(spam_data['length'][spam_data['target'] == 1])) 
step_five()

(71.02362694300518, 138.8661311914324)

<br>

This function has been used to combine new features into the training data:

In [14]:
def add_feature(X, feature_to_add):
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Fitting a Support Vector Classification Model 
using the document-term matrix and an additional feature (**the length of document (number of characters)**

In [15]:
from sklearn.svm import SVC

def step_six():

    vect = TfidfVectorizer(min_df = 5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    #the length of document
    len_train = [len(x) for x in X_train]
    len_test = [len(x) for x in X_test]
    
    #adding features
    X_train_vectorized = add_feature(X_train_vectorized, len_train)
    X_test_vectorized = add_feature(X_test_vectorized, len_test)
    
    # svc model
    model = SVC(probability=True, C=10000)
    model.fit(X_train_vectorized, y_train)
    prediction = model.predict_proba(X_test_vectorized)[:, 1]

    return roc_auc_score(y_test, prediction)

step_six()

0.9963202213809143

In [16]:
# Average number of digits per document for not spam and spam documents

def step_seven():

    spam_data['digit_length'] = spam_data.text.apply(lambda x: len([dig for dig in x if dig.isdigit()]))
    avg_cat_wise = spam_data.groupby('target').mean()
    
    return (avg_cat_wise.digit_length.iloc[0], avg_cat_wise.digit_length.iloc[1])

step_seven()

(0.2992746113989637, 15.759036144578314)

### Fitting a logistic regression model

Fiting and transforming the training data `X_train` using the Tfidf Vectorizer ignoring terms that have a document frequency < **5** and using **word n-grams from n=1 to n=3** 

Using the document-term matrix and the 2 following additional features:
* the length of document (number of characters)
* **number of digits per document**

In [17]:
from sklearn.linear_model import LogisticRegression

def step_eight():

    vect = TfidfVectorizer(min_df = 5, ngram_range = (1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    #counting digits in text
    digits_train = [X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))]
    digits_test = [X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))]
    
    #the length of document
    len_train = [len(x) for x in X_train]
    len_test = [len(x) for x in X_test]
    
    #adding features
    X_train_vectorized = add_feature(X_train_vectorized, len_train)
    X_train_vectorized = add_feature(X_train_vectorized, digits_train)
    X_test_vectorized = add_feature(X_test_vectorized, len_test)
    X_test_vectorized = add_feature(X_test_vectorized, digits_test)
    
    # logistic model
    model = LogisticRegression(C=100)
    model.fit(X_train_vectorized, y_train)
    prediction = model.predict_proba(X_test_vectorized)[:, 1]

    return roc_auc_score(y_test, prediction)

step_eight()

0.9978651342036908

### Average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents

In [18]:
def step_nine():
    return (np.mean(spam_data.loc[spam_data['target']==0,'text'].str.count('\W')), 
            np.mean(spam_data.loc[spam_data['target']==1,'text'].str.count('\W')))
step_nine()

(17.29181347150259, 29.041499330655956)

### V2 of Fitting a logistic regression model 

This time, using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

##### *After the first iteration and new learnings, I updated this model by passing in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This makes the model more robust to spelling mistakes.*

The new set of features : 
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

In [21]:
def step_ten():

    vect = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=[2,5])
    
    X_train_vect = vect.fit_transform(X_train)
    X_test_vect = vect.transform(X_test)
    
    X_train_vect_with_length = add_feature(X_train_vect, [X_train.str.len()
                                                          , X_train.apply(lambda x: len([dig for dig in x if dig.isdigit()]))
                                                          , X_train.str.findall(r'(\W)').str.len()])
    X_test_vect_with_length = add_feature(X_test_vect, [X_test.str.len()
                                                          , X_test.apply(lambda x: len([dig for dig in x if dig.isdigit()]))
                                                          , X_test.str.findall(r'(\W)').str.len()])
    
    clfLG = LogisticRegression(C=100)
    clfLG.fit(X_train_vect_with_length, y_train)
    y_predicted = clfLG.predict_proba(X_test_vect_with_length)[:, 1]
    
    auc_score = roc_auc_score(y_test, y_predicted)

    return (auc_score)

step_ten()

0.996642785596659

## Potential Limitations of the project
* Despite using techniques like document frequency filtering, there's still a risk of overfitting
* The model's effectiveness might be constrained to the language in which it was trained ( English )

I welcome any feedback about how to improve the robustness of my model and address any of the project's limitations other than the listed ones above.

**Thank You for Reviewing This Project**

I appreciate you taking the time to go through my work. Please feel free to reach out if you have any questions, suggestions, or would like to discuss any aspects of this project further.

Best Regards,
